In [1]:
!pip install pandas
!pip install numpy
!pip install datetime
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# 1. GENERATING DUMMY DATA (Simulating your Excel files)
# In a real scenario, you would use: sales_df = pd.read_excel('sales.xlsx')

# Sales History (Item 101 had a promo in Week 1)
sales_data = {
    'Item_id': [101]*14 + [102]*14,
    'Date': pd.to_datetime(['2026-01-01', '2026-01-02', '2026-01-03', '2026-01-04', '2026-01-05', '2026-01-06', '2026-01-07',
                            '2026-01-08', '2026-01-09', '2026-01-10', '2026-01-11', '2026-01-12', '2026-01-13', '2026-01-14']*2),
    'Quantity': [40, 45, 38, 42, 50, 48, 44,  # Week 1 (Promo for 101)
                 5, 7, 4, 6, 5, 8, 6,         # Week 2 (No Promo for 101)
                 10, 12, 11, 9, 13, 10, 11,   # Week 1 (No Promo for 102)
                 10, 11, 12, 10, 11, 13, 12]  # Week 2 (No Promo for 102)
}
sales_df = pd.DataFrame(sales_data)

# Past Promotions
promo_history = pd.DataFrame({
    'Item_id': [101],
    'Start_Date': pd.to_datetime(['2026-01-01']),
    'End_Date': pd.to_datetime(['2026-01-07'])
})

# Current Inventory (WarehouseProduct)
inventory_df = pd.DataFrame({
    'Item_id': [101, 102],
    'Quantity': [10, 50], # Current stock on hand
    'Date': pd.to_datetime(['2026-01-14', '2026-01-14'])
})

# UPCOMING PROMOTIONS (Your Input for next week)
upcoming_promos = pd.DataFrame({
    'Item_id': [102], # Item 102 is going on sale next week!
    'Start_Date': pd.to_datetime(['2026-01-15']),
    'End_Date': pd.to_datetime(['2026-01-21'])
})

# 2. DATA CLEANING
def clean_data(df):
    df = df.drop_duplicates()
    df = df.dropna()
    return df

sales_df = clean_data(sales_df)

# 3. ANALYZE SALES HISTORY (Promotion vs. Non-Promotion)
# Flag sales that happened during a promotion
def check_promo(row):
    is_promo = promo_history[
        (promo_history['Item_id'] == row['Item_id']) & 
        (row['Date'] >= promo_history['Start_Date']) & 
        (row['Date'] <= promo_history['End_Date'])
    ]
    return 1 if not is_promo.empty else 0

sales_df['Is_Promo'] = sales_df.apply(check_promo, axis=1)

# Calculate Average Daily Sales (ADS) for Promo vs Normal
analysis = sales_df.groupby(['Item_id', 'Is_Promo'])['Quantity'].mean().unstack(fill_value=0)
analysis.columns = ['Normal_ADS', 'Promo_ADS']

# 4. PREDICT ORDER FOR NEXT WEEK
# Assume we are ordering for a 7-day period
DAYS_TO_ORDER = 7

def calculate_order(row):
    item = row['Item_id']
    current_stock = inventory_df[inventory_df['Item_id'] == item]['Quantity'].values[0]
    
    # Check if item will be on promo next week
    is_upcoming_promo = item in upcoming_promos['Item_id'].values
    
    # Select the appropriate average daily sales
    if is_upcoming_promo:
        # If we have promo history, use it. If not, use normal + a 300% 'guess' lift
        expected_ads = analysis.loc[item, 'Promo_ADS'] if analysis.loc[item, 'Promo_ADS'] > 0 else analysis.loc[item, 'Normal_ADS'] * 3
    else:
        expected_ads = analysis.loc[item, 'Normal_ADS']
    
    total_needed = expected_ads * DAYS_TO_ORDER
    suggested_order = max(0, total_needed - current_stock)
    
    return pd.Series([expected_ads, total_needed, current_stock, suggested_order])

# Final Result
results = pd.DataFrame({'Item_id': [101, 102]})
results[['Exp_Daily_Sales', 'Total_Needed', 'Current_Stock', 'Suggested_Order']] = results.apply(calculate_order, axis=1)

print("--- Inventory Prediction for Next Week ---")
print(results)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [datetime]2/3 [datetime]
--- Inventory Prediction for Next Week ---
   Item_id  Exp_Daily_Sales  Total_Needed  Current_Stock  Suggested_Order
0      101         5.857143          41.0           10.0             31.0
1      102        33.214286         232.5           50.0            182.5
